### This document is used to preprocess the large files we are dealing with, for visualization purposes, and to make it a bit simpler to visualize

In [31]:
import pandas as pd

#### Gather patients with admission table

In [32]:
parent_path = 'D:\skolaÅr5\MasterThesis\mimic-iv-1.0'

In [33]:
df_patients = pd.read_csv(parent_path + '\core\patients.csv')

In [34]:
df_admissions = pd.read_csv(parent_path + r'\core\admissions.csv')

In [35]:
df_patadm = pd.merge(df_patients, df_admissions, on='subject_id')

### Select n patients with m admissions

In [36]:
n_patients = 1
m_admissions = 5

In [37]:
admperpat = df_patadm.groupby(['subject_id']).size().reset_index(name='count')
more_than_nappointments = admperpat[admperpat['count'] > m_admissions]

In [49]:
npatients = more_than_nappointments.sample(n_patients)['subject_id'].tolist()

In [50]:
npatients

[18018694]

#### Based on the selected patients, gather diagnose data

In [40]:
codes = pd.read_csv(parent_path + r'/hosp/d_icd_diagnoses.csv')
pat_diagnoses = pd.read_csv(parent_path + r'/hosp/diagnoses_icd.csv')

In [41]:
pat_diagnoses = pat_diagnoses[pat_diagnoses['subject_id'].isin(npatients)]

In [42]:
dropcol = ['icd_version_x', 'icd_version_y', 'seq_num']
df_diagpatcodes = pat_diagnoses.merge(codes, on='icd_code', how='inner').drop(columns=dropcol)

In [43]:
df_diagpatcodes

,subject_id,hadm_id,icd_code,long_title
0,11521280,23926006,25000,Diabetes mellitus without mention of complicat...
1,11521280,28862443,25000,Diabetes mellitus without mention of complicat...
2,11521280,29899490,25000,Diabetes mellitus without mention of complicat...
3,11521280,23926006,3051,Tobacco use disorder
4,11521280,26057130,3051,Tobacco use disorder
...,...,...,...,...
119,11521280,21081223,E8889,Other specified metabolic disorders
120,11521280,21081223,E8339,Fall on stairs or ladders in water transport i...
121,11521280,21081223,E8339,Other disorders of phosphorus metabolism
122,11521280,21081223,Z8546,Personal history of malignant neoplasm of pros...


#### Based on the selected patients, gather chart-events

##### Since the file is very large, we will use spark to process the data

In [44]:
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,col
import pyspark.sql.functions as F
import numpy as np

In [45]:
spark = SparkSession \
            .builder \
            .appName("Python Spark SQL basic example") \
            .config("spark.some.config.option", "some-value") \
            .getOrCreate()

In [46]:
spark

In [19]:
path = r'C:\Users\Johan\Desktop\mimicIV'

In [22]:
df_chart_events = spark.read.load(path + r'\chartevents.csv', format="csv", sep=",", inferSchema="true", header="true")

In [51]:
df_chart_events_filtered = df_chart_events.filter(df_chart_events.subject_id.isin(npatients))

In [52]:
df_chart_events_filtered.count()

0

In [ ]:
df_chart_events_filtered.show()

In [28]:
columns_to_drop = ['stay_id', 'warning', 'valuenum']

In [29]:
df_chart_events_filtereddroped = df_chart_events_filtered.drop(*columns_to_drop)

In [30]:
df_pandas = df_chart_events_filtereddroped.toPandas()

KeyboardInterrupt: 